In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
print(os.listdir("."))
os.chdir("drive/MyDrive/Colab Notebooks/")
print(os.listdir("."))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['.config', 'drive', 'sample_data']
['x_train.npy', 'y_test.npy', 'y_train.npy', 'x_test.npy', 'Training.ipynb']


In [3]:
import numpy as np
import keras
#print(keras.__version__)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.models import load_model

In [4]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [5]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_train= x_train/255
x_test = x_test/255

In [6]:
y_test = keras.utils.to_categorical(y_test, 36)
y_train = keras.utils.to_categorical(y_train, 36)

In [10]:
model = Sequential()

#input size is 32 by 32 and filter is grayscale,
#we want to get 32 filters so here 32 filters of size (3, 3) will be used which gives 32 filter maps
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1), data_format = 'channels_last'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(36, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)         

In [11]:
#Compile the model
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)

model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [14]:
hist = model.fit(x_train, y_train,batch_size=32,epochs=10,verbose=1,validation_data=(x_test, y_test))

Epoch 1/10
915/915 [==============================] - 10s 11ms/step - loss: 0.0604 - accuracy: 0.9789 - val_loss: 0.0769 - val_accuracy: 0.9759
Epoch 2/10
915/915 [==============================] - 10s 11ms/step - loss: 0.0559 - accuracy: 0.9793 - val_loss: 0.0787 - val_accuracy: 0.9766
Epoch 3/10
915/915 [==============================] - 10s 11ms/step - loss: 0.0564 - accuracy: 0.9795 - val_loss: 0.0813 - val_accuracy: 0.9747
Epoch 4/10
915/915 [==============================] - 10s 11ms/step - loss: 0.0619 - accuracy: 0.9780 - val_loss: 0.0748 - val_accuracy: 0.9774
Epoch 5/10
915/915 [==============================] - 10s 11ms/step - loss: 0.0525 - accuracy: 0.9813 - val_loss: 0.0801 - val_accuracy: 0.9766
Epoch 6/10
915/915 [==============================] - 10s 10ms/step - loss: 0.0561 - accuracy: 0.9800 - val_loss: 0.0784 - val_accuracy: 0.9764
Epoch 7/10
915/915 [==============================] - 10s 10ms/step - loss: 0.0574 - accuracy: 0.9797 - val_loss: 0.0735 - val_accuracy:

In [15]:
model.save('model' + '.h5')